In [48]:
import uuid
import pandas as pd
from reportlab.lib.pagesizes import landscape, A4
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from openpyxl import load_workbook
import smtplib
import os
import shutil
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

In [49]:
# Añadimos nuestra fuente
pdfmetrics.registerFont(TTFont('Belleza', 'fuentes/Belleza.ttf'))

In [50]:
# Leer los nombres de los destinatarios desde el archivo Excel
# Lee el archivo Excel y conviértelo en un DataFrame
df = pd.read_excel('participantes/participantes.xlsx')

# Crea una lista vacía para almacenar los destinatarios
destinatarios = []

# Itera sobre las filas del DataFrame
for index, row in df.iterrows():
    # Crea un diccionario para cada destinatario
    destinatario = {"nombre": row[1], "correo": row[2]}  # El nombre está en la segunda columna (índice 1)
    # Agrega el destinatario a la lista
    destinatarios.append(destinatario)

# Muestra la lista de destinatarios
# print(destinatarios)

In [ ]:
destinatario["nombre"]

In [53]:
# Crear los documentos PDF para cada destinatario
for destinatario in destinatarios:
    # Obtener los datos del destinatario actual
    nombre_destinatario = destinatario["nombre"]

    # Generar el código de identificación único
    codigo = str(uuid.uuid4())

    # Crear el documento PDF y definir su tamaño
    pdf = canvas.Canvas(f"certificados_participantes/{nombre_destinatario}.pdf", pagesize=landscape(A4))

    # Agregar la imagen de fondo
    pdf.drawImage("Fondo/fondo_participante.png", 0, 0, width=842, height=595)

    # Agregar el nombre del destinatario
    pdf.setFillColorRGB(0.16, 0.50, 0.51) # Color
    pdf.setFont("Belleza", 36) # Fuente
    pdf.drawCentredString(421, 360, f"{nombre_destinatario}")

    # Agregar el código de identificación
    pdf.setFillColorRGB(0, 0, 0) # Color
    pdf.setFont("Helvetica", 10) # Fuente
    pdf.drawString(18, 12, f"Código: OE-{codigo}")

    # Cerrar el documento PDF
    pdf.save()

In [30]:
# Enviar correos
# Datos del servidor SMTP de Gmail
smtp_server = 'smtp.gmail.com'
smtp_port = 587  # Puerto para TLS
smtp_username = 'Orientación Económica'
smtp_user = 'lpariapazap@gmail.com'
smtp_password = 'uvroxxickdzfwuzx'

# Destinatarios de los correos electrónicos


# Archivo a enviar
archivo = f"{nombre_destinatario}.pdf"

# Configura el servidor SMTP de Gmail
server = smtplib.SMTP(smtp_server, smtp_port)
server.starttls()
server.login(smtp_user, smtp_password)

# Envía el correo electrónico a cada destinatario
for destinatario in destinatarios:
    nombre = destinatario['nombre']
    correo = destinatario['correo']
    mensaje = f'Hola {nombre}, este es un mensaje de prueba con un archivo adjunto.'
    remitente = smtp_user
    asunto = 'Mensaje de prueba con archivo adjunto'

    # Crea el mensaje de correo electrónico
    msg = MIMEMultipart()
    msg['From'] = smtp_username
    msg['To'] = correo
    msg['Subject'] = asunto
    msg.attach(MIMEText(mensaje))

    # Adjunta el archivo al mensaje
    with open(archivo, "rb") as f:
        archivo_adjunto = MIMEApplication(f.read(), _subtype=os.path.splitext(archivo)[1][1:])
        archivo_adjunto.add_header('Content-Disposition', 'attachment', filename=os.path.basename(archivo))
        msg.attach(archivo_adjunto)

    # Envía el mensaje de correo electrónico
    cuerpo = msg.as_string()
    server.sendmail(remitente, correo, cuerpo)

# Cierra la conexión con el servidor SMTP
server.quit()

(221,
 b'2.0.0 closing connection m23-20020a67ed57000000b00423e2fdeab6sm114350vsp.34 - gsmtp')